In [2]:
import requests
from copy import copy

# Establish baseline
r = requests.get("http://localhost:5000")
ciphertext_baseline = r.cookies["authtoken"]
ciphertext_bytes_baseline = bytearray.fromhex(ciphertext_baseline)

In [3]:
# Determine block size
ciphertext_bytes = copy(ciphertext_bytes_baseline)

def determine_block_size(ciphertext_bytes: bytearray):
    count = 0
    for i in range(len(ciphertext_bytes_baseline)):
        ciphertext_bytes[i] ^= 0x01
        cookie = {"authtoken": ciphertext_bytes.hex()}
        r = requests.get("http://localhost:5000/quote", cookies=cookie)
        
        if (r.content != b'No quote for you!'):
            return count
        
        count += 1

    return -1

block_size = determine_block_size(ciphertext_bytes)
print(f'Block size: {block_size}')

Block size: 16


Note when looking at the ciphertext we use the fact that the last block wont change other blocks since it's the last to find the block size. This means that changes in the cipher here shouldn't result in a change in the rest of the cipher when decoding and therefore it should be able to decode still.

A change in the other parts should change more of the ciphertext resulting in a decode failure.

In [84]:
def oracle(block: bytearray, iv: bytearray, ciphertext_bytes: bytearray):
    block = int.to_bytes(int.from_bytes(block) ^ int.from_bytes(iv), length=block_size)
    start_end = block_count * block_size - block_size * (i_block + 1)
    end_start = block_count * block_size - block_size * i_block

    print(start_end)
    print(end_start)

    ciphertext_bytes = ciphertext_bytes[:start_end] + block + ciphertext_bytes[end_start:]

    cookie = {"authtoken": ciphertext_bytes.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)

    print(r.content)
        
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or r.content == b'No quote for you!' or r.content == b'Zero-length input cannot be unpadded'):
        return False
    else:
        return True
        



zero_iv = [0] * block_size

ciphertext_bytes = copy(ciphertext_bytes_baseline)

block_count = len(ciphertext_bytes) // block_size

block = ciphertext_bytes[-16:]
i_block = 0

flag_found = False

for i_block_byte in range(block_size):
    padding_iv = [i_block_byte ^ b for b in zero_iv]
    print(f"Block byte: {i_block_byte}")

    for i in range(256):
        # Guess 1 bytes of the padding iv at a time.
        padding_iv[-(i_block_byte + 1)] = i
        iv = bytes(padding_iv) 

        # Test the padding iv on the block
        if (oracle(block, iv, ciphertext_bytes)):
            if i_block_byte == 0:
                padding_iv[-2] ^= 1
                iv = bytes(padding_iv)

                if not oracle(block, iv, ciphertext_bytes):
                    continue

            flag_found = True
            print(f"Found correct byte at: {i}")
            break

    if (not flag_found):
        pass#raise Exception("no valid padding byte found")
    else:
        flag_found = False
    zero_iv[-(i_block_byte + 1)] = i ^ i_block_byte


Block byte: 0
96
112
b'No quote for you!'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b"'utf-8' codec can't decode byte 0xcd in position 80: invalid continuation byte"
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'PKCS#7 padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.'
96
112
b'Padding is incorrect.

In [6]:
import requests
import curses
from collections.abc import Callable 
from copy import copy

def oracle(iv: bytearray, block: bytearray):
    # Create token with the iv and block
    token = iv + block

    # Create request and attach cookie with token
    cookie = {"authtoken": token.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)

    # Check response
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or  r.content == b'Zero-length input cannot be unpadded'): #r.content == b'No quote for you!' or
        return False
    else:
        return True

def process_block(block: bytearray, oracle, i_block: int):
    zero_iv = [0] * block_size
    found_result = False

    for i_block_byte in range(1, block_size + 1):
        padding_iv = [i_block_byte ^ b for b in zero_iv]#[(i_block_byte + 1) ^ b for b in zero_iv]

        select_str = "" 
        for i in range(block_size + 1, 1, -1):
            if (i_block_byte == i):
                select_str += "^^"
            else:
                select_str += "  "
        
        for i in range(0, 256):
            padding_iv[-i_block_byte] = i
            iv = bytes(padding_iv)

            pre_str = f'Block {i_block}: '

            print(f'{pre_str}{iv.hex()}\n{len(pre_str)*" "}{select_str}', end=f'\r')
        
            if (oracle(iv, block)):
                # Check the case where we haven't changed anything yet.
                # The reason for this check is that it will not throw a padding error and think
                # it's correct without doing anything. Therefore we change the byte in front and check if its correct.
                if (i_block_byte == 1):
                    padding_iv[-2] ^= 1
                    iv = bytes(padding_iv)
                    if (not oracle(iv, block)):
                        continue
                found_result = True
                break
            
        if (found_result == False):
            print("Did not find the correct values.")

        found_result = False
        zero_iv[-i_block_byte] = i ^ i_block_byte

    return zero_iv

# Create blocks
blocks = [bytearray()] * block_count
for i_block in range(block_count):
    blocks[i_block] = ciphertext_bytes[block_size * i_block:len(ciphertext_bytes)-(block_size * (block_count - i_block - 1))]
    print(f'len: {len(blocks[i_block])}; {blocks[i_block].hex()}')

# Do attack
def attack(iv, ct, oracle, print_status_callback: Callable[[], None]):
    msg = iv + ct
    blocks = [msg[i:i+block_size] for i in range(0, len(msg), block_size)]
    result = b''

    iv = blocks[0]

    for i_block in range(1, len(blocks)):
        block = blocks[i_block]
        dec = process_block(block, oracle, i_block, print_status_callback)
        pt = bytes(iv_byte ^ dec_byte for iv_byte, dec_byte in zip(iv, dec))
        result += pt
        iv = block

    return result
    
def print_status(i_block: int, iv: bytearray):
    pass

# Establish baseline
r = requests.get("http://localhost:5000")
ciphertext_baseline = r.cookies["authtoken"]
ciphertext_baseline_bytes = bytearray.fromhex(ciphertext_baseline)

iv = ciphertext_baseline_bytes[:16]
ct = ciphertext_baseline_bytes[16:]


stdscr = curses.initscr()
#curses.echo()
#curses.nocbreak()
#curses.endwin()


print(iv)
print("Ciphertext:", ct)
print("Launching attack...")

result = attack(iv, ct, oracle, print_status)

r = requests.get("http://localhost:5000/quote", cookies={"authtoken": result.hex()})
print(r.content)
print(result.decode(encoding="utf-8"))

#plaintext = unpad(result, AES.block_size)
#print("Recovered plaintext:", plaintext)
#print("Decoded:", b64decode(plaintext).decode('ascii'))


len: 16; 195f8abcb1406aaf814db8b7953fbcfc
len: 16; b6f37849e6ff2471d40519387cb96b5a
len: 16; 4e35aa8d2c7a21a0470fe2595547a612
len: 16; b018523fe34a1601cee1102ff22f6cbc
len: 16; c3db5124d01e4313645edd794c11a12b
len: 16; 94680a22c76bd9a2ad3893ad2c7ca1db
len: 16; 9de6fc009f0fd11551315d6bcb4ea9b0
bytearray(b'w\xa4\xb0\x91\x103i\x88\xda\x19\x97\xd5x\xef[\xc0')
Ciphertext: bytearray(b'\xd5y\x85t\xcb\x81\xae\x1e18\x14`\xa4M\xfc\x9d\xad\xc2X\xdd@*\x03&\xce\x93\xfe\x1a+U\xd2\r\x85\x1b\xb7\x17/\x0c.\x1c\xa6\x8ebm.\xe9KD<\xc9\xae\x00,\x96\x06a\xb3-m\xc2z\x80d\x07\x07\xd3\x95W\x08\xb4\x1f6\x17Q(\x1c\xee/\x1d\xda\xae\xe5\xe0\x91\x19\xe7i\x99[\xd4\x06H\x9d\xd5r\xd8')
Launching attack...
Block 1: 01010101010101010101010101010100
Block 1: 01010101010101010101010101010101
Block 1: 01010101010101010101010101010102
Block 1: 01010101010101010101010101010103
Block 1: 01010101010101010101010101010104
Block 1: 01010101010101010101010101010105
Block 1: 01010101010101010101010101010106
Block 1: 010101010101010

In [48]:
import random
import os

from Crypto.Cipher import AES  # requires PyCryptodome
from Crypto.Util.Padding import pad, unpad


class Challenge:
    _strings = (
        b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93"
    )

    def __init__(self):
        self._key = os.urandom(16)

    def get_string(self):
        """This is the first function described by Challenge 17."""
        string = random.choice(self._strings)
        cipher = AES.new(self._key, AES.MODE_CBC)
        ct = cipher.encrypt(pad(string, AES.block_size))
        return cipher.iv, ct

    def check_padding(self, iv, ct):
        """This is the second function described by Challenge 17."""
        cipher = AES.new(self._key, AES.MODE_CBC, iv)
        pt = cipher.decrypt(ct)
        try:
            unpad(pt, AES.block_size)
        except ValueError:  # raised by unpad() if padding is invalid
            return False
        return True

In [56]:
from base64 import b64decode
service = Challenge()
iv, ct = service.get_string()
print(iv)
print("Ciphertext:", ct)
print("Launching attack...")

result = attack(iv, ct, service.check_padding)
plaintext = unpad(result, AES.block_size)
print("Recovered plaintext:", plaintext)
print("Decoded:", b64decode(plaintext).decode('ascii'))

b'?\xaa"\x9a\x0c\xcei]\xe4]M\x91\x8fi_1'
Ciphertext: b"\xdd\xd1\xac&G\xfc\x03Ne\xab\xe09E\xd9\x9e>\x8c'h\r\xaf<\xd9\xe7\x0f\xddd\x8c\xe7D\x90J\xbd\xdc\x12\xed\xd4]\x9a\xee\x97[\x9c\x81\x96\xf1\xa9\x82\xb9\xedl\xd0\x17\xb8\x94\xc7\x07\xb5\x023R\x01\x9a\xebW\xfc\xb7\xc6\x83\x16\x1bz\xe7\xe3a\xcdC }j"
Launching attack...
87
46
88
210
228
29
2
185
45
34
129
77
224
109
225
98
15
169
141
32
74
164
228
36
43
97
176
47
76
154
153
169
59
171
0
128
238
55
157
102
222
145
78
193
48
36
107
208
246
199
192
243
236
192
27
214
145
165
17
145
147
77
189
207
230
148
14
90
58
8
190
3
194
146
191
23
224
95
149
243
Recovered plaintext: b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw=='
Decoded: 000002Quick to the point, to the point, no faking
